In [1]:
import pandas as pd
import glob, os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

2022-04-07 10:21:31.180395: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-07 10:21:31.180411: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# gather csv data in the form
path = r'/home/qan/Efrei/data_eng/scala-spark-kafka-project/ressources/reports/reportsFiles' # use your path
all_files = glob.glob(path + "/*.csv")


df_from_each_file = (pd.read_csv(f, sep=';', on_bad_lines='skip', usecols=range(1,7)) for f in all_files)
final_df = pd.concat(df_from_each_file, ignore_index=True)
final_df.head()

 
li = []

for i, filename in enumerate(all_files):
    df = pd.read_csv(filename, names=["peacewatcherID", "Peacewatcherlat", "Peacewatcherlon", "CitizensID",
      "CitizensPeaceScore", "Words", "datetimeReport"], index_col=None, header=0, sep=';', on_bad_lines='skip')
    df.reset_index(drop=True, inplace=True)
    li.append(df)
 

In [3]:
frame = pd.concat(li, axis=0, ignore_index=True)
frame.head()

,peacewatcherID,Peacewatcherlat,Peacewatcherlon,CitizensID,CitizensPeaceScore,Words,datetimeReport
0,1,-28.143780,-117.629234,"[328,79,580,252,803,849,347,791,372,859,337,60...","[87.0,40.0,91.0,96.0,99.0,40.0,6.0,58.0,52.0,9...","[autocuiseur,copyright,occident,éblouissement,...",6-4-2022 17:22
1,2,38.554995,176.992991,"[199,202,685,183,493,988,673,789,31,593,240,714]","[49.0,62.0,53.0,60.0,45.0,24.0,17.0,1.0,10.0,5...","[anime,maladroit,délibération,salir,tentée,int...",6-4-2022 17:22
2,3,-34.723582,-86.099011,"[88,704,334,320,792,508,754,58,779,854,540,686...","[87.0,64.0,25.0,69.0,29.0,79.0,24.0,61.0,16.0,...","[refusée,Lemire,festivités,bienveillant,profon...",6-4-2022 17:22
3,4,10.585115,112.759830,"[73,648,603,496,750,85,406,665,80,902,182,140,...","[3.0,47.0,57.0,53.0,75.0,32.0,10.0,18.0,25.0,3...","[coté,mauvais,chaude journée,angoisser,quasi,c...",6-4-2022 17:22
4,5,-0.687541,77.800509,"[288,620,390,236,184,491,501,962,475,106,187,256]","[42.0,7.0,57.0,36.0,5.0,99.0,77.0,66.0,53.0,8....","[afflux,discorde,aisés,sensibiliser,supporters...",6-4-2022 17:22


In [4]:
frame.to_csv("reports.csv", index=False)

## Using transformers and Flaubert model to predict the sentiment of each report

In [13]:
# get custom tokenizer for french langage
loaded_tokenizer = AutoTokenizer.from_pretrained('flaubert/flaubert_large_cased')

# get Flaubert model for sentiment analysis
loaded_model = AutoModelForSequenceClassification.from_pretrained("DemangeJeremy/4-sentiments-with-flaubert")

# create the pipeline
nlp = pipeline('sentiment-analysis', model=loaded_model, tokenizer=loaded_tokenizer)

Some weights of the model checkpoint at DemangeJeremy/4-sentiments-with-flaubert were not used when initializing FlaubertForSequenceClassification: ['transformer.position_ids']
- This IS expected if you are initializing FlaubertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaubertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Analysis of the sentiment of a report

In [26]:
report = frame['Words'][80]
analysis = nlp(report)
print(report)

[Dresde,barrique,bourrin,analyses,visé,arrêtées,marron,spirituel,abandon,jusant,notation,parlera,musarder,ivoire,chu,négociants,camé,musicale,vociférer,Virgin,salaud,approbation,crabe,escomptés,devront,ressemblent,continent,reconduction,ressort,esquisser,rattrapage,Agathe,guise de,entretenue,communication,design,Tokyo,habiter,dérobée,sentant,objections,représentée,plaine,manager,mentionné,ignoble,protestations,bénéficié,épouser,vifs,équitable,militants,chaque,ironie,mécénat,jeep,vinaigre,partiel,immédiate,propos,molécules,opposer,remarquait,réplique,cauchemar,prévenir]


In [31]:
print('Sentiment label: ', analysis[0]['label'])
print('Sentiment score: ', analysis[0]['score'])

Sentiment label:  MIXED
Sentiment score:  0.748498260974884


In [46]:
sub = df.sample(n=100)
sub.shape

(100, 7)

In [47]:
sub['ReportSentiment'] = sub.apply (lambda row: nlp(row['Words']), axis=1)

In [48]:
pd. set_option('display.max_rows', 500)
sub

,peacewatcherID,Peacewatcherlat,Peacewatcherlon,CitizensID,CitizensPeaceScore,Words,datetimeReport,ReportSentiment
59,60,-5.396665,-23.137303,"[867,197,550,659,440,846,193,284,506,548,491,9...","[67.0,74.0,76.0,17.0,7.0,72.0,44.0,80.0,74.0,7...","[scaphandrier,fasciné,animateur,laissés,admini...",6-4-2022 17:28,"[{'label': 'NEGATIVE', 'score': 1.0}]"
74,75,2.272047,78.393081,"[42,426]","[79.0,0.0]","[expirer,voulait,spacieux,conceptions,migratio...",6-4-2022 17:28,"[{'label': 'NEGATIVE', 'score': 1.0}]"
36,37,-10.694978,-111.929554,"[405,868,540,121,543,859,100,459,18,774,128,43...","[85.0,48.0,15.0,97.0,17.0,77.0,57.0,47.0,22.0,...","[bergerie,conjugaison,passa,pensait,concédé,im...",6-4-2022 17:28,"[{'label': 'NEGATIVE', 'score': 1.0}]"
95,96,64.539691,-172.659651,"[124,126,20,507,658,825,95,146,205,399,773]","[82.0,91.0,15.0,34.0,67.0,99.0,88.0,90.0,60.0,...","[carcéral,inquiétants,accumulé,moustache,parut...",6-4-2022 17:28,"[{'label': 'MIXED', 'score': 0.6960018873214722}]"
31,32,-78.390798,105.287366,"[455,578,773]","[20.0,79.0,63.0]","[grimper,radicalisation,il,surchauffe,avant-h...",6-4-2022 17:28,"[{'label': 'NEGATIVE', 'score': 1.0}]"
32,33,-34.434232,-78.033744,"[419,272,892,737,197,341,232,914,949,925,177,1...","[41.0,38.0,27.0,18.0,64.0,63.0,37.0,83.0,92.0,...","[Rudolf,changement,mielleux,tournesol,élaborat...",6-4-2022 17:28,"[{'label': 'NEGATIVE', 'score': 1.0}]"
51,52,23.861625,-160.259569,"[533,523,605,283,487,440,887,396,318,31,737,13...","[46.0,9.0,54.0,56.0,90.0,38.0,5.0,4.0,35.0,41....","[cafetière,bush,sonorité,irrésolu,rotatif,dieu...",6-4-2022 17:28,"[{'label': 'POSITIVE', 'score': 0.445892691612..."
65,66,16.205445,-27.660712,"[956,402,164,400]","[62.0,79.0,12.0,45.0]","[sidérurgistes,bottin,planer,moutard,soit,effr...",6-4-2022 17:28,"[{'label': 'NEGATIVE', 'score': 1.0}]"
89,90,9.673517,156.532884,"[371,341,798,535,575,439,115,17,309,633,433,37...","[21.0,68.0,80.0,3.0,15.0,13.0,15.0,98.0,49.0,7...","[former,conciliation,allait,remparts,crois,dur...",6-4-2022 17:28,"[{'label': 'NEGATIVE', 'score': 1.0}]"
15,16,-74.843371,-83.047541,"[187,251,865,480,256,186,587,410,38,302,324,49...","[69.0,26.0,27.0,68.0,0.0,75.0,53.0,96.0,69.0,1...","[balance,âpreté,maximum,vôtre,excellentes,bota...",6-4-2022 17:28,"[{'label': 'NEGATIVE', 'score': 1.0}]"
